# Résumé automatique de documents financiers

Objectif : transformer rapidement un **rapport financier** (annuel, trimestriel, comptes, bilan, annexes) en un **résumé clair et chiffré**.

# Résumé automatique de documents financiers (PDF)


### PARTIE 1 — Préparation (imports, .env)
- Charger les librairies nécessaires  
- Lire les variables depuis `.env` (clé API, chemin du PDF, modèle OpenAI)  
- Vérifier la configuration minimale  

### PARTIE 2 — Lecture du PDF (avec repères par page)
- Ouvrir le fichier PDF avec **PyMuPDF**  
- Extraire le texte page par page  
- Ajouter un séparateur clair `=== [PAGE X] ===`  
- Nettoyer le texte et le tronquer si trop long  

### PARTIE 3 — Consignes orientées FINANCE
- Définir le rôle : *analyste financier*  
- Exiger un **résumé structuré** (Société, Résumé exécutif, Chiffres clés, Analyse, Références internes)  
- Imposer la règle : **ne pas inventer de chiffres** (mettre `non précisé` si absent)  
- Demander un tableau de 6 à 12 indicateurs maximum  

### PARTIE 4 — Appel à l’API OpenAI (Responses)
- Envoyer deux messages (system + user) à l’API  
- Utiliser `client.responses.create(...)`  
- Récupérer le texte résumé depuis `response.output_text`  

### PARTIE 5 — Affichage du résumé 
- Afficher le résumé brut dans la console  
- Afficher le résumé en Markdown (plus lisible dans Jupyter)  
- Ajouter un rappel : **toujours vérifier les chiffres et pages d’origine**  


In [ ]:
#Prérequis (exécuter une seule fois dans le notebook) 
!pip install openai PyMuPDF python-dotenv

# PARTIE 1 — PRÉPARATION (imports, .env)

In [ ]:
import os
import fitz # PyMuPDF
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
import os, pathlib
print("Dossier courant :", pathlib.Path.cwd())

# Chercher et charger le .env (remonte les dossiers si besoin)
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv(filename=".env", usecwd=True)
env_path

In [ ]:

load_dotenv(dotenv_path=env_path, override=True)              # Charge explicitement le fichier .env et écrase les variables existantes si nécessaire
  
api_key = os.getenv("OPENAI_API_KEY")    
print(api_key[:4] + "..." + "****")  
  


# Partie 2: Lecture du PDF (avec repères par page)

In [ ]:

chemin_pdf = "./data/teslafinancialreport.pdf"

In [ ]:

pdf = fitz.open(chemin_pdf)

texte = ""
for i, page in enumerate(pdf, start=1):
    texte_page = page.get_text()
    texte += f"\n\n=== [PAGE {i}] ===\n" + texte_page.strip()

print(texte)

In [ ]:
# Nettoyage léger : retirer espaces inutiles
texte = "\n".join(l.strip() for l in texte.splitlines())
texte

In [ ]:
# Option de sécurité : éviter d'envoyer un texte excessivement long
LONGUEUR_MAX = 120_000 # augmentez si nécessaire
if len(texte) > LONGUEUR_MAX:
    texte = texte[:LONGUEUR_MAX]

In [ ]:
# Aperçu rapide (facultatif) :
print("Aperçu du début du texte :\n")
print(texte[:800])

# PARTIE 3 — Consignes (format de sortie)

In [ ]:
consignes = (
"Tu es analyste financier. On te fournit le texte d'un document financier\n"
"(rapport annuel, trimestriel, comptes, bilan, annexes).\n\n"
"Produis une synthèse **précise et chiffrée** en Markdown selon ce cadre :\n\n"
"- **Société / Période / Devise** : (si repérable)\n"
"- **Résumé exécutif (5–8 lignes)** : activité, faits marquants, contexte\n"
"- **Chiffres clés** (tableau) :\n"
" | Indicateur | Valeur | Évolution/Contexte | Période | Page |\n"
" |---|---:|---|---|---:|\n"
" (exemples : Chiffre d'affaires, EBIT/EBITDA, Résultat net, Marge, FCF, CAPEX,\n"
" Dette nette, Trésorerie, NPL/Coût du risque pour banque, CET1, LCR/NSFR, etc.)\n"
"- **Analyse** :\n"
" - Performance (croissance, marges, cash)\n"
" - Structure financière (dette, liquidité)\n"
" - Risques & incertitudes (marché, réglementation, change)\n"
" - Outlook / Guidance (si communiqué)\n"
"- **Références internes** : pages/sections à relire\n\n"
"Exigences :\n"
"- **N'invente aucun chiffre**. Si une valeur n'apparaît pas clairement : `non précisé`.\n"
"- Cite la **Page** d'origine quand c'est possible (repère `=== [PAGE X] ===`).\n"
"- 6 à 12 **indicateurs quantitatifs** maximum (les plus utiles).\n"
"- Reste concis : 200–350 mots hors tableau."
)

PARTIE 4 — Appel API OpenAI (Responses)

In [ ]:
client = OpenAI(api_key=api_key)

modele = "gpt-4o-mini"

reponse = client.responses.create(
    model=modele,
    input=[
        {"role": "system", "content": consignes},
        {"role": "user", "content": texte},
    ],
)

# Partie 5: Affichage du résumé (markdown)

In [ ]:
resume = reponse.output_text


print("\n=== RÉSUMÉ FINANCIER GÉNÉRÉ ===\n")
print(resume)

In [ ]:
display(Markdown(resume))



# Partie 6: Questions interactives sur le PDF

Cette partie permet de poser des **questions spécifiques** sur le contenu du PDF
et d'obtenir des réponses précises, sourcées si possible (référence de page).

Exemples de questions :
- Quel est le chiffre d’affaires 2023 ?
- Quelle est la marge nette du groupe ?
- Quels sont les principaux risques identifiés ?


In [ ]:
# Question de l'utilisateur (à modifier librement)
ma_question = "Quel est le résultat net ?"

In [ ]:
# Consignes pour le modèle : répondre uniquement avec le texte fourni
consignes_questions = (
    "Tu es analyste financier. On te donne un extrait de rapport financier. "
    "Réponds uniquement à la question posée, sans inventer de données. "
    "Si la réponse n'est pas claire dans le texte, écris : 'non précisé'. "
    "Quand c'est possible, indique aussi la page d'origine (repère '=== [PAGE X] ===')."
)

In [ ]:
# Appel API
reponse_question = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": consignes_questions},
        {"role": "user", "content": f"Question : {ma_question}\n\nTexte PDF :\n{texte}"},
    ],
)


In [ ]:
# Afficher la réponse
print("\n=== RÉPONSE À LA QUESTION ===\n")
print(reponse_question.output_text)
display(Markdown(reponse_question.output_text))

In [ ]:
# ================================
# BOUCLE INTERACTIVE SUR LE PDF
# ================================

consignes_questions = (
    "Tu es analyste financier. On te donne le texte d’un rapport financier. "
    "Réponds uniquement à la question posée, sans inventer de données. "
    "Si la réponse n’est pas claire dans le texte, écris : 'non précisé'. "
    "Quand c’est possible, indique aussi la page d’origine (repère '=== [PAGE X] ===')."
)

while True:
    # Demander une question à l'utilisateur
    ma_question = input("\nPosez votre question (ou tapez 'bye' pour quitter) : ")
    
    # Condition de sortie
    if ma_question.strip().lower() == "bye":
        print("👋 Fin de l’interaction. Merci d’avoir utilisé l’application !")
        break
    
    # Appel API pour répondre
    reponse_question = client.responses.create(
        model=modele,
        input=[
            {"role": "system", "content": consignes_questions},
            {"role": "user", "content": f"Question : {ma_question}\n\nTexte PDF :\n{texte}"},
        ],
    )
    
    # Afficher la réponse
    print("\n=== RÉPONSE ===\n")
    print(reponse_question.output_text)

# Rappel des bonnes pratiques

> **Note :** Vérifiez toujours les chiffres affichés et leurs pages d'origine.  
> En cas d'ambiguïté dans le PDF, remplacez par *« non précisé »* et prenez  
> quelques minutes pour confirmer dans le document source.
